Ref: https://github.com/microsoft/ai-agents-for-beginners/blob/main/13-agent-memory/13-agent-memory.ipynb

In [1]:
!pip install -q mem0ai

In [2]:
import json
import os
from typing import Annotated, List, Dict, Any
from datetime import datetime
import uuid

from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

# Azure AI Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchField,
    VectorSearchAlgorithmMetric
)

# Mem0
from mem0 import Memory

# Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatHistory
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread


/Users/sulbhajain/opt/anaconda3/envs/azure_agent_py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2371: PydanticDeprecatedSince211: The `__get_pydantic_core_schema__` method of the `BaseModel` class is deprecated. If you are calling `super().__get_pydantic_core_schema__` when overriding the method on a Pydantic model, consider using `handler(source)` instead. However, note that overriding this method on models can lead to unexpected side effects. Deprecated in Pydantic V2.11 to be removed in V3.0.
  schema = annotation_get_schema(source, get_inner_schema)


In [ ]:
# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Use a recent API version

# Azure AI Search Configuration
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")

# Index names
travel_index_name = "travel-hotels"
memory_index_name = "mem0-memories"

In [4]:
# Initialize search clients
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Create travel data index if it doesn't exist
travel_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="name", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchableField(name="location", type=SearchFieldDataType.String),
    SearchableField(name="amenities", type=SearchFieldDataType.String),
    SimpleField(name="price_per_night", type=SearchFieldDataType.Double),
    SimpleField(name="rating", type=SearchFieldDataType.Double),
    SearchableField(name="tags", type=SearchFieldDataType.String, collection=True)
]

travel_index = SearchIndex(name=travel_index_name, fields=travel_fields)

try:
    index_client.get_index(travel_index_name)
    print(f"✅ Index '{travel_index_name}' already exists")
except:
    index_client.create_index(travel_index)
    print(f"✅ Created index '{travel_index_name}'")

# Initialize search client for travel data
travel_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=travel_index_name,
    credential=AzureKeyCredential(search_api_key)
)

ServiceRequestError: URL has an invalid label.

In [5]:
# Add sample travel data
sample_hotels = [
    {
        "id": "1",
        "name": "Le Meurice Paris",
        "description": "Luxury palace hotel with Michelin-starred dining and views of the Tuileries Garden",
        "location": "Paris, France",
        "amenities": "Spa, Michelin Restaurant, Concierge, Room Service, Fitness Center",
        "price_per_night": 850,
        "rating": 4.8,
        "tags": ["luxury", "romantic", "historic", "fine-dining", "spa"]
    },
    {
        "id": "2",
        "name": "Four Seasons Maui",
        "description": "Beachfront resort with world-class spa and family-friendly activities",
        "location": "Maui, Hawaii",
        "amenities": "Beach Access, Kids Club, Multiple Pools, Spa, Golf Course",
        "price_per_night": 695,
        "rating": 4.7,
        "tags": ["beach", "family-friendly", "resort", "spa", "golf"]
    },
    {
        "id": "3",
        "name": "Aman Tokyo",
        "description": "Minimalist luxury hotel with panoramic city views and traditional onsen",
        "location": "Tokyo, Japan",
        "amenities": "Onsen, City Views, Fine Dining, Spa, Business Center",
        "price_per_night": 780,
        "rating": 4.9,
        "tags": ["luxury", "business", "spa", "city", "minimalist"]
    },
    {
        "id": "4",
        "name": "Hotel Sacher Vienna",
        "description": "Historic hotel home of the original Sachertorte with elegant rooms",
        "location": "Vienna, Austria",
        "amenities": "Historic Cafe, Concierge, Accessible Rooms, Pet-Friendly",
        "price_per_night": 420,
        "rating": 4.6,
        "tags": ["historic", "accessible", "pet-friendly", "cultural", "cafe"]
    },
    {
        "id": "5",
        "name": "Fairmont Whistler",
        "description": "Ski-in/ski-out resort with family suites and mountain views",
        "location": "Whistler, Canada",
        "amenities": "Ski Access, Family Suites, Heated Pool, Kids Programs",
        "price_per_night": 380,
        "rating": 4.5,
        "tags": ["ski", "family-friendly", "mountain", "resort", "accessible"]
    }
]

# Upload hotels to search index
travel_search_client.upload_documents(documents=sample_hotels)
print(f"✅ Uploaded {len(sample_hotels)} hotels to search index")


NameError: name 'travel_search_client' is not defined

In [6]:
mem0_config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_deployment,
            "temperature": 0.2,
            "max_tokens": 1500,
            "azure_kwargs": {
                "azure_deployment": azure_openai_deployment,
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": search_service_endpoint.split("//")[1].split(".")[0],
            "api_key": search_api_key,
            "collection_name": "mem0",
            "embedding_model_dims": 1536
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-ada-002",  # Your embedding deployment name
            "azure_kwargs": {
                "azure_deployment": "text-embedding-ada-002",  # Update if different
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    }
}

# Initialize Mem0
memory = Memory.from_config(mem0_config)
# Test the memory system
print("🧪 Testing Mem0 setup...")
test_messages = [
    {"role": "user", "content": "I prefer luxury hotels with spa services."},
    {"role": "assistant", "content": "I'll remember you prefer luxury hotels with spa services for future recommendations."}
]
memory.add(test_messages, user_id="test_user",
           metadata={"category": "preferences"})
test_memories = memory.get_all(user_id="test_user")
print(f"✅ Mem0 test successful! Found {len(test_memories)} memories")


ServiceRequestError: URL has an invalid label.

In [7]:
class TravelBookingPlugin:
    """Plugin for searching hotels and managing user travel preferences"""

    def __init__(self, search_client: SearchClient, memory: Memory):
        self.search_client = search_client
        self.memory = memory

    @kernel_function(
        description="Search for hotels based on criteria like location, amenities, or tags"
    )
    def search_hotels(
        self,
        query: Annotated[str, "Search query for hotels (location, amenities, etc.)"],
        max_results: Annotated[int, "Maximum number of results to return"] = 3
    ) -> Annotated[str, "List of hotels matching the search criteria"]:
        """Search for hotels in the travel database"""
        results = self.search_client.search(
            search_text=query,
            top=max_results,
            include_total_count=True
        )

        hotels = []
        for result in results:
            hotels.append({
                "name": result["name"],
                "location": result["location"],
                "description": result["description"],
                "price_per_night": result["price_per_night"],
                "rating": result["rating"],
                "amenities": result["amenities"],
                "tags": result["tags"]
            })

        return json.dumps(hotels, indent=2)

    @kernel_function(
        description="Store user travel preferences and important information in memory"
    )
    def store_user_preference(
        self,
        user_id: Annotated[str, "User identifier"],
        preference: Annotated[str,
                              "User preference or information to remember"]
    ) -> Annotated[str, "Confirmation of stored preference"]:
        """Store user preferences in Mem0 memory"""
        print(f"DEBUG: Storing preference for {user_id}: {preference}")

        try:
            # Simply add the preference to memory
            self.memory.add(preference, user_id=user_id)
            return f"✅ Stored: {preference}"
        except Exception as e:
            return f"❌ Error storing preference: {str(e)}"
        
    @kernel_function(
        description="Get all stored preferences for a user"
    )
    def get_user_preferences(
        self,
        user_id: Annotated[str, "User identifier"]
    ) -> Annotated[str, "All user preferences and memories"]:
        """Get all memories for a specific user"""
        print(f"DEBUG: Getting all preferences for {user_id}")

        try:
            # Get all memories for the user
            results = self.memory.get_all(user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No preferences found for user {user_id}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    memories.append(memory_text)
                else:
                    memories.append(str(result))

            return f"User preferences for {user_id}:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR getting preferences: {str(e)}")
            return f"No preferences found for user {user_id}"


    @kernel_function(
        description="Search user's memories for relevant information"
    )
    def search_memories(
        self,
        user_id: Annotated[str, "User identifier"],
        query: Annotated[str,
                         "What to search for (e.g., 'family vacation', 'dietary restrictions')"]
    ) -> Annotated[str, "Relevant memories"]:
        """Search user memories using Mem0"""
        print(f"DEBUG: Searching memories for {user_id} with query: '{query}'")

        try:
            # Let Mem0 handle the search and ranking
            results = self.memory.search(query, user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No memories found for query: {query}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    # Include relevance score if available
                    score = result.get('score', None)
                    if score:
                        memories.append(
                            f"{memory_text} (relevance: {score:.2f})")
                    else:
                        memories.append(memory_text)
                else:
                    memories.append(str(result))

            return "Relevant memories:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR: {str(e)}")
            return "No memories found."
      

In [8]:
# Create the kernel
kernel = Kernel()

# Add Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
)
kernel.add_service(chat_service)

# Create and add the travel booking plugin
travel_plugin = TravelBookingPlugin(travel_search_client, memory)
kernel.add_plugin(
    plugin_name="TravelBooking",
    plugin=travel_plugin
)

# Create the travel agent
# Create the travel agent
travel_agent = ChatCompletionAgent(
    service=chat_service,
    name="TravelBookingAssistant",
    instructions="""
    You are a personalized travel booking assistant with memory.
    
    WORKFLOW:
    1. When a user asks for help, search their memories using search_memories() with a relevant query
    2. Use the memories to personalize your response
    3. Store any new preferences they mention using store_user_preference()
    4. When the users is booking a new trip, first retrieve the users general travel preferences of the user by creating queries for hotels, dietary restrictions, location, amenities and budget. THEN use search_hotels() to find suitable options.
    5. Do not recommend hotels that are over budget. 
    
    IMPORTANT: For ALL memory operations (search_memories and store_user_preference), 
    you MUST use user_id='sarah_johnson_123' exactly as written.

    Example queries:
    - User asks about booking a trip → search_memories(query="preferences")
    - User asks about booking a trip → search_memories(query="dietary restrictions")
    - User asks about booking a trip → search_memories(query="location")
    - User asks about booking a trip → search_memories(query="amenities")
    - User asks about booking a trip → search_memories(query="budget")

    Always acknowledge what you found in their memories when responding.""",
    plugins=[travel_plugin]
)

NameError: name 'travel_search_client' is not defined

In [ ]:
def display_message(role: str, content: str, color: str = "#2E8B57", emoji: str = ""):
    """Display a message with nice formatting"""
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 15px 20px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.05); 
        border-radius: 8px;
    '>
        <strong style='color: {color}; font-size: 16px;'>{emoji} {role}:</strong><br>
        <div style='margin-top: 10px; white-space: pre-wrap; font-size: 14px; line-height: 1.6;'>{content}</div>
    </div>
    """
    display(HTML(html))

def display_memory_operation(operation: str, details: str, color: str = "#9370DB"):
    """Display memory operations for educational purposes"""
    html = f"""
    <div style='
        margin: 5px 20px;
        padding: 10px 15px;
        background: rgba(147, 112, 219, 0.1);
        border: 1px solid {color};
        border-radius: 6px;
        font-family: monospace;
        font-size: 13px;
    '>
        <strong style='color: {color};'>🧠 Memory {operation}:</strong>
        <div style='margin-top: 5px; color: #555;'>{details}</div>
    </div>
    """
    display(HTML(html))

def display_function_call(function_name: str, args: dict, result: str = None):
    """Display function calls for transparency"""
    html = f"""
    <details style='margin: 5px 20px; padding: 10px; background: rgba(0, 123, 255, 0.05); border: 1px solid #007BFF; border-radius: 6px;'>
        <summary style='cursor: pointer; font-weight: bold; color: #007BFF;'>⚙️ Function Call: {function_name}</summary>
        <div style='margin-top: 10px; font-family: monospace; font-size: 12px;'>
            <div><strong>Arguments:</strong> {json.dumps(args, indent=2)}</div>
    """
    if result:
        html += f"<div style='margin-top: 10px;'><strong>Result:</strong><pre style='background: #f8f8f8; padding: 8px; border-radius: 4px; overflow-x: auto;'>{result}</pre></div>"
    html += "</div></details>"
    display(HTML(html))
